In [56]:
import pandas as pd
import numpy as np
import pyodbc
import math
from ast import literal_eval
from tqdm import tqdm
import os
from configparser import ConfigParser
import statistics
from scipy import stats as s

# from connector.mosaicio import MosaicioConnector

In [57]:
# data_ind_U  = pd.read_csv(r"/notebooks/notebooks/Synthetic_data_Clustered_individual_U.csv")
# #uncomment to run for org
# # data_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\Synthetic_data_Clustered_organization_U.csv")

# ind_data_thresholds_U = pd.read_csv(r"/notebooks/notebooks/threshold_individual_U.csv")
# #uncomment to run for org
# # org_data_thresholds_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\threshold_organization_U.csv")

# weight_table_ind_U = pd.read_csv(r"/notebooks/notebooks/weight_individual_U.csv")
# #uncomment to run for org
# # weight_table_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\weight_organization_U.csv")

In [58]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [59]:
cursor.execute('''INSERT INTO  FINCORE_DB.Fincore_Analytics.RISK_PERSON_ANALYTICAL_RULES
select
A.PERSON_MASTER_ID,
A.RULE_NO,
case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end Risk_score,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_rules a 
inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG b 
on a.PERSON_MASTER_ID = b.PERSON_MASTER_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.PEER_GROUP_ID and a.RULE_NO = c.RULE_NO
where a.RISK_SCORE_STATIC is null''')
conn.commit()

In [60]:
risk_an_per = pd.read_sql_query('''select 
A.PERSON_MASTER_ID,
A.RULE_NO,
A.RULE_VALUE,
case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end RISK_SCORE,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_rules a 
inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG b 
on a.PERSON_MASTER_ID = b.PERSON_MASTER_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.PEER_GROUP_ID and a.RULE_NO = c.RULE_NO
where a.RISK_SCORE_STATIC is null''',conn)

In [61]:
# for i in tqdm(range(10)):
#     df_cal = pd.read_csv((r"cal_") + str(i) + ".csv")
#     risk_an_per = risk_an_per.append(df_cal, ignore_index=True)

In [62]:
risk_an_per.drop_duplicates(inplace=True)
risk_an_per.to_csv(r"/data/risk_an_per.csv")

In [63]:
risk_an_per = pd.read_csv(r"/data/risk_an_per.csv")
risk_an_per.drop(columns = ['Unnamed: 0'], inplace = True)
risk_an_per.dropna(inplace =  True)

In [64]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [65]:
risk_an_per.drop_duplicates(inplace=True)

In [66]:
risk_an_per

,PERSON_MASTER_ID,RULE_NO,RULE_VALUE,RISK_SCORE,CREATE_DATE
0,1,P002001001,4.0,10,2023-02-20 13:27:14.900
1,1,P002001002,1.0,1,2023-02-20 13:27:14.900
2,1,P002001004,4.0,10,2023-02-20 13:27:14.900
3,1,P002001007,4.0,10,2023-02-20 13:27:14.900
4,1,P002001008,1.0,1,2023-02-20 13:27:14.900
...,...,...,...,...,...
2415,423,P002001024,1.0,1,2023-02-20 13:27:14.900
2416,423,P002001027,1.0,1,2023-02-20 13:27:14.900
2417,423,P002001030,1.0,1,2023-02-20 13:27:14.900
2418,423,P002001033,1.0,1,2023-02-20 13:27:14.900


In [67]:
# for ind,row in tqdm(risk_an_per.iterrows()):
#     conn.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_ANALYTICAL_RULES] (PERSON_MASTER_ID, RULE_NO, THRESHOLD_FLG)
#     values (?,?,?)''',(row['PERSON_MASTER_ID'],row['RULE_NO'],row['RISK_SCORE']))
# #     conn.execute('''INSERT INTO [Fincore_Analytics].[RISK_PERSON_ANALYTICAL_RULES] (PERSON_MASTER_ID, RULE_NO, RULE_VALUE,THRESHOLD_FLG) VALUES (?, ?, ?,?)''',(row['PERSON_MASTER_ID'],row['RULE_NO'],row['RULE_VALUE'],row['THRESHOLD_FLG']))
# #     conn.execute('INSERT INTO [Fincore_Analytics].[RISK_PERSON_ANALYTICAL_RULES] (PERSON_MASTER_ID, RULE_NO, RULE_VALUE,THRESHOLD_FLG) VALUES (?, ?, ?,?)', result)
# conn.commit()
# conn.close()


0it [00:00, ?it/s]


IntegrityError: ('23000', "[23000] [FreeTDS][SQL Server]Violation of PRIMARY KEY constraint 'XPK_RISK_PERSON_ANALYTICAL_RULES'. Cannot insert duplicate key in object 'Fincore_Analytics.RISK_PERSON_ANALYTICAL_RULES'. The duplicate key value is (1, P002001001). (2627) (SQLExecDirectW)")

In [68]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [69]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [70]:
risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with (nolock)", conn)

In [71]:
# Make a list of all the Static rules

static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Individual') & (risk_lkp['RISK_TYPE'] != 'Auto High Rules ')]['RULE_NO'].to_list()
static_rules

['P002001013',
 'P003001001',
 'P003001004',
 'P003001007',
 'P003001008',
 'P003001009',
 'P003001010']

In [72]:
# Read risk_person_summary_stg_rules for the static rules
if len(static_rules) == 1:
    stat_rule_df = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('{}')".format(static_rules[0]),conn)
    
elif(len(static_rules)== 0):
    stat_rule_df = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('')",conn)
    
else:
    static_rules = tuple(static_rules)
    stat_rule_df = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in" + str (static_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [73]:
stat_rule_df[~pd.isnull(stat_rule_df['RISK_SCORE_STATIC'])]

,PERSON_MASTER_ID,RULE_NO,RULE_VALUE,RISK_SCORE_STATIC,CREATE_DATE
0,1,P002001013,IN,8,2023-02-20
1,1,P003001004,Current Account,5,2023-02-20
2,1,P003001007,2022-11-15,10,2023-02-20
3,1,P003001008,High,8,2023-02-20
4,1,P003001009,2015-06-20,3,2023-02-20
...,...,...,...,...,...
923,423,P002001013,IN,8,2023-02-20
924,423,P003001004,Savings Account,6,2023-02-20
925,423,P003001008,High,8,2023-02-20
926,423,P003001009,2019-06-20,7,2023-02-20


In [74]:
# The stat_rule_df is converted to a data frame which matches the shape of the risk_an_per

stat_rule_df = stat_rule_df[['PERSON_MASTER_ID','RULE_NO','RISK_SCORE_STATIC']]
stat_rule_df.drop_duplicates(inplace=True)

In [75]:
# stat_rule_df['THRESHOLD_FLG'] = 10
#Dont need to insert list based(static) rules in analytical table
stat_rule_df.rename(columns={'RISK_SCORE_STATIC':'RULE_VALUE'},inplace=True)
stat_rule_df['THRESHOLD_FLG'] = stat_rule_df['RULE_VALUE']

In [76]:
# Appending the data of static and non static rules to calculate the final risk score

risk_an_per = risk_an_per.append(stat_rule_df)

In [77]:
temp = risk_an_per.groupby(['PERSON_MASTER_ID','RULE_NO']).size().reset_index()

In [78]:
risk_an_per.to_csv(r"/data/risk_an_per_with_static.csv")

In [79]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [80]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [81]:
risk_per_analytical = risk_an_per
risk_weight =  pd.read_sql_query("SELECT * FROM FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES with (nolock)",conn)

#Changing the structure of the table
risk_per_analytical = risk_per_analytical.pivot(index='PERSON_MASTER_ID', columns='RULE_NO', values=['THRESHOLD_FLG'])
#col[1] contains only the first name of the column
risk_per_analytical.columns = [col[1] for col in risk_per_analytical.columns.values]
# risk_per_analytical = risk_per_analytical.reset_index()
risk_per_analytical = risk_per_analytical.reset_index()



In [82]:
#weight columns contains all the rule columns
weight_col = risk_per_analytical.columns
weight_col = [col for col in weight_col if col.startswith('P0')]
#Risk_weight contains the weightage of the weight_col
risk_weight = risk_weight[risk_weight['RULE_NO'].isin(weight_col)]['WEIGHTAGE']
risk_weight
# risk_weight.pivot(columns='RULE_NO', values=['WEIGHTAGE'])

50     10
52     10
54     10
56     10
68     10
69     10
72     10
73     10
74     10
79     10
80     10
81     10
82     10
83     10
91     10
93     10
94     10
95     10
96     10
97     10
98     10
99     10
136    10
137    10
138    10
139    10
140    10
142    10
144    10
146    10
147    10
148    10
149    10
150    10
209    10
Name: WEIGHTAGE, dtype: int64

In [83]:
#creating the dataframe of the weight_col to make comparison easier
rule_weight_df = pd.DataFrame(data = [risk_weight.values],columns = weight_col)
rule_weight_df

,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,...,P002003003,P002003005,P002003006,P002003008,P003001001,P003001004,P003001007,P003001008,P003001009,P003001010
0,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10


In [84]:
weighted_avg_denom = pd.read_sql_query("select sum(WEIGHTAGE) from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES with (nolock) where PERSON_TYPE = 'Individual' and IS_ACTIVE_FLAG = 1 and RISK_TYPE <> 'Auto High Rules ' ",conn)

weighted_avg_denom = weighted_avg_denom.iloc[0,0]
weighted_avg_denom

420

In [85]:
risk_person_summary_stg_rules = pd.read_sql_query("Select * from fincore_db.fincore_analytics.risk_person_summary_stg_rules with (nolock)",conn)

weights = pd.read_sql_query("Select RULE_NO,WEIGHTAGE from fincore_db.fincore_analytics.risk_lkp_rules with (nolock)",conn)

risk_person_summary_stg_rules = risk_person_summary_stg_rules.merge(weights,how = 'inner',on = 'RULE_NO')

risk_person_summary_stg_rules.drop(columns=  ['RULE_VALUE','RISK_SCORE_STATIC','CREATE_DATE'],inplace = True)

risk_person_summary_stg_rules.drop_duplicates(inplace = True)

risk_person_summary_stg_rules.drop(columns= ['RULE_NO'],inplace = True)

denominator = risk_person_summary_stg_rules.groupby('PERSON_MASTER_ID').sum()

risk_per_analytical = risk_per_analytical.merge(denominator,how = 'inner',on = 'PERSON_MASTER_ID')

In [86]:
risk_per_analytical

,PERSON_MASTER_ID,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,...,P002003005,P002003006,P002003008,P003001001,P003001004,P003001007,P003001008,P003001009,P003001010,WEIGHTAGE
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,3.0,3.0,160
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,NaN,NaN,90
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,NaN,NaN,90
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,NaN,NaN,90
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,NaN,NaN,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,8.0,7.0,7.0,170
335,420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,8.0,7.0,7.0,170
336,421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,8.0,7.0,7.0,170
337,422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,8.0,7.0,7.0,170


In [87]:
# #here calc risk score
# per_analytical_cols = risk_per_analytical.columns.to_list()
# #per_analytical will contain all the columns of risk_per_analytical in addition to below columns
# per_analytical_cols.extend(['RULE_LIST','RISK_SCORE'])
# per_analytical = []
# count = 1
# for ind,row in tqdm(risk_per_analytical.iterrows()):
#     if count%10000 == 0:
#         print(row['PERSON_MASTER_ID'],' HERE')
#     count += 1
#     l1 = []

#     if False: 
# #     ((math.isnan(row['P002001001']) == True) & (math.isnan(row['P002001010']) == True) & (math.isnan(row['P002001011']) == True) & (math.isnan(row['P002001018']) == True & (math.isnan(row['P002001019']) == True) & (math.isnan(row['P002001025']) == True) & (math.isnan(row['P002001026']) == True))) == True: #compare here for nan, in this case risk_score will be null
#         data = row[weight_col]
#         l1.append(row['PERSON_MASTER_ID'])
#         l1.extend(data.values)
#         rule_list = "None" #rule_list will be empty in this case
#         l1.append(rule_list)
#         risk_score = -1 #nan risk score
#         l1.append(risk_score)
# #         print(row)
# #         print(risk_score)
# #         print(rule_list)
# #         l1 containg the entire row including the risk score and appending it with per_analytical list
#         per_analytical.append(l1)
# #         break
#     else:
        
#         data = row[weight_col] * rule_weight_df 
#         l1.append(row['PERSON_MASTER_ID'])
#         l1.extend(data.values[0])
        
#         #Rule_list will include only those columns that has non null values
#         rule_list = [val for val in row.keys() if val.startswith('P0') and ~np.isnan(row[val]) and row[val] > 0 ]
#         rule_list = "_".join(rule_list)
#         num_rules = [val for val in data.values[0] if ~np.isnan(val) and val != 0]
#         l1.append(rule_list)
#         #calculating risk score by summing the row and dividing by count of non null rules
# #         risk_score = (np.sum(num_rules))/(len(num_rules) * 10)
#         risk_score = (np.sum(num_rules))/(weighted_avg_denom)
#         l1.append(risk_score)
#         per_analytical.append(l1)
#         print(l1)
        

In [88]:
risk_per_analytical

,PERSON_MASTER_ID,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,...,P002003005,P002003006,P002003008,P003001001,P003001004,P003001007,P003001008,P003001009,P003001010,WEIGHTAGE
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,3.0,3.0,160
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,NaN,NaN,90
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,NaN,NaN,90
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,NaN,NaN,90
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,10.0,8.0,NaN,NaN,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,8.0,7.0,7.0,170
335,420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,8.0,7.0,7.0,170
336,421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,8.0,7.0,7.0,170
337,422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,8.0,7.0,7.0,170


In [89]:
#here calc risk score
per_analytical_cols = risk_per_analytical.columns.to_list()
#per_analytical will contain all the columns of risk_per_analytical in addition to below columns
per_analytical_cols.extend(['RULE_LIST','RISK_SCORE'])
per_analytical_cols.remove('WEIGHTAGE')
per_analytical = []
count = 1
for ind,row in tqdm(risk_per_analytical.iterrows()):
    if count%10000 == 0:
        print(row['PERSON_MASTER_ID'],' HERE')
    count += 1
    l1 = []

    if False: 
#     ((math.isnan(row['P002001001']) == True) & (math.isnan(row['P002001010']) == True) & (math.isnan(row['P002001011']) == True) & (math.isnan(row['P002001018']) == True & (math.isnan(row['P002001019']) == True) & (math.isnan(row['P002001025']) == True) & (math.isnan(row['P002001026']) == True))) == True: #compare here for nan, in this case risk_score will be null
        data = row[weight_col]
        l1.append(row['PERSON_MASTER_ID'])
        l1.extend(data.values)
        rule_list = "None" #rule_list will be empty in this case
        l1.append(rule_list)
        risk_score = -1 #nan risk score
        l1.append(risk_score)
#         print(row)
#         print(risk_score)
#         print(rule_list)
#         l1 containg the entire row including the risk score and appending it with per_analytical list
        per_analytical.append(l1)
#         break
    else:
        
        data = row[weight_col] * rule_weight_df 
        l1.append(row['PERSON_MASTER_ID'])
        l1.extend(data.values[0])
        
        #Rule_list will include only those columns that has non null values
        rule_list = [val for val in row.keys() if val.startswith('P0') and ~np.isnan(row[val]) and row[val] > 0 ]
        rule_list = "_".join(rule_list)
        num_rules = [val for val in data.values[0] if ~np.isnan(val) and val != 0]
        l1.append(rule_list)
        #calculating risk score by summing the row and dividing by count of non null rules
#         risk_score = (np.sum(num_rules))/(len(num_rules) * 10)
        risk_score = (np.sum(num_rules))/row['WEIGHTAGE']
        l1.append(risk_score)
        per_analytical.append(l1)
        print(l1)
        

167it [00:00, 837.94it/s]

[1.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 80.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 50.0, 100.0, 80.0, 30.0, 30.0, 'P002001013_P003001004_P003001007_P003001008_P003001009_P003001010', 2.3125]
[2.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 80.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 50.0, 100.0, 80.0, nan, nan, 'P002001013_P003001004_P003001007_P003001008', 3.4444444444444446]
[3.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 80.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 50.0, 100.0, 80.0, nan, nan, 'P002001013_P003001004_P003001007_P003001008', 3.4444444444444446]
[4.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 80.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 50.0, 100.0, 80.0, nan, nan, 'P002001013_P003001004_P003001007_P00300100

339it [00:00, 857.58it/s]

[325.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 80.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 50.0, nan, 80.0, 70.0, 70.0, 'P002001013_P003001004_P003001008_P003001009_P003001010', 1.9444444444444444]
[326.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '', 0.0]
[327.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '', 0.0]
[328.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '', 0.0]
[329.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 80.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 60.0

In [90]:
data_ind_U = pd.read_sql_query('''select PERSON_MASTER_ID,PEER_GROUP_ID from fincore_db.fincore_analytics.risk_person_summary_stg with (nolock) ''',conn)

In [91]:
#creating a new dataframe
risk_per_analytical_modified = pd.DataFrame(data = per_analytical,columns=per_analytical_cols)

In [92]:
#Merging it with data_ind_U to get the peer_group_id
risk_per_analytical_modified_df = pd.merge(data_ind_U[['PERSON_MASTER_ID','PEER_GROUP_ID']],risk_per_analytical_modified,on = 'PERSON_MASTER_ID')

In [93]:
risk_per_analytical_modified_df.drop_duplicates(inplace=True)

In [94]:
data = risk_per_analytical_modified_df[['PERSON_MASTER_ID','PEER_GROUP_ID','RISK_SCORE','RULE_LIST']]
data.to_csv(r"/data/risk_person.csv")

In [95]:
# data = pd.read_csv(r"/data/risk_person.csv")

In [1]:
try:
    data.drop(columns = ['Unnamed: 0'],axis = 1,inplace= True)
except:
    pass
    

In [97]:
data['RISK_SCORE'].fillna(0,inplace=True)

In [98]:
data['RULE_LIST'].fillna('None',inplace=True)

data.to_csv(r"/data/risk_person_final.csv")

In [99]:
data.head()

,PERSON_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,RULE_LIST
0,1,Government Sector_0,2.312500,P002001013_P003001004_P003001007_P003001008_P0...
1,2,Government Sector_0,3.444444,P002001013_P003001004_P003001007_P003001008
2,3,Government Sector_0,3.444444,P002001013_P003001004_P003001007_P003001008
3,4,Government Sector_0,3.444444,P002001013_P003001004_P003001007_P003001008
4,5,Government Sector_0,3.100000,P002001013_P003001004_P003001007_P003001008


In [100]:
data['RISK_SCORE'] = round(data['RISK_SCORE'])

In [101]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [102]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [103]:
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()
# conn = pyodbc.connect(config_data["connection"])
count = 1
#UPDATING RISK_PERSON
# for ind,row in risk_per_analytical_modified_df.iterrows():
for ind,row in tqdm(data.iterrows()):
    if count%10000 == 0:
        print(row['PERSON_MASTER_ID'],' HERE')
        conn.commit()
    count += 1
    conn.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_PERSON (PERSON_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,RISK_RULE_LIST,CREATE_DATE)
    values ({},'{}',{},'{}',{})'''.format(row['PERSON_MASTER_ID'],row['PEER_GROUP_ID'],row['RISK_SCORE'],row['RULE_LIST'],'Getdate()'))
    
#     conn.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_PERSON (PERSON_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,RISK_RULE_LIST)
#     values (?,?,?,?)''',(row['PERSON_MASTER_ID'],row['PEER_GROUP_ID'],row['RISK_SCORE'],row['RULE_LIST']))

conn.commit()
conn.close()

339it [00:00, 817.46it/s]


In [104]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [105]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [106]:
risk_person =  pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON] with (nolock)",conn)

In [107]:
%store -r no_of_records

In [108]:
no_of_records

339

In [109]:
%store -r job_id

job_id

106

In [110]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()


In [111]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [112]:
cursor.execute(f'''Update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),JOB_STATUS = 'Success',NO_OF_RECORD = {no_of_records},SOURCE_MAX_DATE =  (select max(update_date) from FINCORE_DB.Fincore.person_master) where  JOB_RUN_ID ={job_id}''')

cursor.commit()
conn.close()